<a href="https://colab.research.google.com/github/cn23070/cn23070.github.io/blob/main/Module4EoMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3  # SQLite library for database operations.
from hashlib import sha256  # Provides hash functions, used for password hashing and checksum generation.
from datetime import datetime  # For managing and formatting timestamps.
from cryptography.fernet import Fernet  # Provides encryption/decryption functionality.
import os  # For file path operations and checking file existence.
import getpass  # For securely handling password inputs.

# Define Classes

class UserRole:
    ADMIN = "admin"
    REGULAR_USER = "regular_user"

class User:
    def __init__(self, user_id, username, password_hash, role):
        self.user_id = user_id
        self.username = username
        self.password_hash = password_hash
        self.role = role

    def set_password(self, password):
        """Hashes the password and stores it securely."""
        self.password_hash = sha256(password.encode()).hexdigest()

    def check_password(self, password):
        """Verifies the entered password against the stored hash."""
        return self.password_hash == sha256(password.encode()).hexdigest()

    def get_role(self):
        """Returns the role of the user."""
        return self.role

    def get_user_id(self):
        """Returns the user ID."""
        return self.user_id

    def get_username(self):
        """Returns the username."""
        return self.username

    def get_password_hash(self):
        """Returns the password hash."""
        return self.password_hash

class Administrator(User):
    def create_artefact(self, artefact, db_manager):
        """Allows the admin to create and store an artefact."""
        db_manager.store_artefact(artefact)

    def delete_artefact(self, artefact_id, db_manager):
        """Allows the admin to delete an artefact."""
        db_manager.delete_artefact(artefact_id)

    def view_artefact(self, artefact_id, db_manager):
        """Allows the admin to view an artefact."""
        return db_manager.retrieve_artefact(artefact_id)

    def view_logs(self, db_manager):
        """Allows the admin to view logs."""
        db_manager.view_logs()

class RegularUser(User):
    def upload_artefact(self, artefact, db_manager):
        """Allows the user to upload their own artefact."""
        if artefact.owner_id == self.user_id:
            db_manager.store_artefact(artefact)

    def modify_artefact(self, artefact, db_manager):
        """Allows the user to modify their own artefact."""
        existing_artefact = db_manager.retrieve_artefact(artefact.artefact_id)
        if existing_artefact and existing_artefact.owner_id == self.user_id:
            db_manager.update_artefact(artefact)

    def view_artefact(self, artefact_id, db_manager):
        """Allows the user to view an artefact."""
        artefact = db_manager.retrieve_artefact(artefact_id)
        if artefact and (artefact.owner_id == self.user_id or self.get_role() == UserRole.ADMIN):
            return artefact
        return None

class Artefact:
    def __init__(self, artefact_id, title, content, owner_id):
        self.artefact_id = artefact_id
        self.title = title
        self.content = content
        self.checksum = self.generate_checksum(content)
        self.encrypted_content = None
        self.timestamps = self.create_timestamp()
        self.owner_id = owner_id

    def generate_checksum(self, data):
        """Generates a checksum to verify data integrity."""
        return sha256(data).hexdigest()

    def encrypt_content(self, encryption_manager):
        """Encrypts the content using the EncryptionManager."""
        self.encrypted_content = encryption_manager.encrypt(self.content)

    def decrypt_content(self, encryption_manager):
        """Decrypts the content using the EncryptionManager."""
        return encryption_manager.decrypt(self.encrypted_content)

    def create_timestamp(self):
        """Creates a timestamp for the artefact."""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class EncryptionManager:
    def __init__(self):
        """Initializes the encryption manager with a secure key."""
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

    def encrypt(self, data):
        """Encrypts the provided data."""
        return self.cipher.encrypt(data)

    def decrypt(self, data):
        """Decrypts the provided data."""
        return self.cipher.decrypt(data)

class ChecksumManager:
    @staticmethod
    def generate_checksum(data):
        """Generates a checksum for the provided data."""
        return sha256(data).hexdigest()

    @staticmethod
    def verify_checksum(data, checksum):
        """Verifies the checksum of the provided data."""
        return sha256(data).hexdigest() == checksum

class TimestampManager:
    @staticmethod
    def create_timestamp():
        """Creates a current timestamp."""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    @staticmethod
    def update_timestamp():
        """Updates the timestamp."""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class DatabaseManager:
    def __init__(self, db_name="secure_enclave.db"):
        """Initializes the database manager and connects to the SQLite database."""
        self.db_name = db_name
        self.connect_to_db()

    def connect_to_db(self):
        """Connects to the SQLite database."""
        self.db_connection = sqlite3.connect(self.db_name, timeout=10)
        self.db_cursor = self.db_connection.cursor()
        self.create_tables()

    def create_tables(self):
        """Creates the necessary tables if they don't exist."""
        self.db_cursor.execute('''CREATE TABLE IF NOT EXISTS Users (
                                    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    username TEXT UNIQUE NOT NULL,
                                    password_hash TEXT NOT NULL,
                                    role TEXT NOT NULL)''')

        self.db_cursor.execute('''CREATE TABLE IF NOT EXISTS Artefacts (
                                    artefact_id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    title TEXT NOT NULL,
                                    content BLOB NOT NULL,
                                    checksum TEXT NOT NULL,
                                    encrypted_content BLOB NOT NULL,
                                    timestamps TEXT NOT NULL,
                                    owner_id INTEGER NOT NULL,
                                    FOREIGN KEY(owner_id) REFERENCES Users(user_id))''')

        self.db_cursor.execute('''CREATE TABLE IF NOT EXISTS Logs (
                                    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    action TEXT NOT NULL,
                                    user_id INTEGER,
                                    artefact_id INTEGER,
                                    timestamp TEXT NOT NULL,
                                    FOREIGN KEY(user_id) REFERENCES Users(user_id),
                                    FOREIGN KEY(artefact_id) REFERENCES Artefacts(artefact_id))''')

        self.db_connection.commit()

    def store_user(self, user):
        """Stores a user in the database."""
        self.db_cursor.execute('''INSERT INTO Users (username, password_hash, role)
                                  VALUES (?, ?, ?)''',
                               (user.username, user.password_hash, user.role))
        self.db_connection.commit()

    def store_artefact(self, artefact):
        """Stores an artefact in the database."""
        self.db_cursor.execute('''INSERT INTO Artefacts (title, content, checksum, encrypted_content, timestamps, owner_id)
                                  VALUES (?, ?, ?, ?, ?, ?)''',
                               (artefact.title, artefact.content, artefact.checksum,
                                artefact.encrypted_content, artefact.timestamps, artefact.owner_id))
        self.db_connection.commit()

    def retrieve_artefact(self, artefact_id):
        """Retrieves an artefact by its ID."""
        self.db_cursor.execute('SELECT * FROM Artefacts WHERE artefact_id = ?', (artefact_id,))
        row = self.db_cursor.fetchone()
        if row:
            return Artefact(row[0], row[1], row[2], row[6])
        return None

    def update_artefact(self, artefact):
        """Updates an existing artefact."""
        self.db_cursor.execute('''UPDATE Artefacts
                                  SET title = ?, content = ?, checksum = ?, encrypted_content = ?, timestamps = ?
                                  WHERE artefact_id = ?''',
                               (artefact.title, artefact.content, artefact.checksum,
                                artefact.encrypted_content, artefact.timestamps, artefact.artefact_id))
        self.db_connection.commit()

    def delete_artefact(self, artefact_id):
        """Deletes an artefact by its ID."""
        self.db_cursor.execute('DELETE FROM Artefacts WHERE artefact_id = ?', (artefact_id,))
        self.db_connection.commit()

    def get_user_by_username(self, username):
        """Retrieves user details by username."""
        self.db_cursor.execute('SELECT * FROM Users WHERE username = ?', (username,))
        row = self.db_cursor.fetchone()
        if row:
            return User(row[0], row[1], row[2], row[3])
        return None

    def view_logs(self):
        """Retrieve and print all logs."""
        self.db_cursor.execute('SELECT * FROM Logs')
        logs = self.db_cursor.fetchall()
        for log in logs:
            print(log)

    def close(self):
        """Closes the database connection."""
        if self.db_connection:
            self.db_connection.close()

class Logger:
    def __init__(self, db_manager):
        """Initializes the logger with a reference to the database manager."""
        self.db_manager = db_manager

    def log_event(self, event, user_id=None, artefact_id=None):
        """Logs an event to the database."""
        timestamp = TimestampManager.create_timestamp()
        self.db_manager.db_cursor.execute('''INSERT INTO Logs (action, user_id, artefact_id, timestamp)
                                            VALUES (?, ?, ?, ?)''',
                                         (event, user_id, artefact_id, timestamp))
        self.db_manager.db_connection.commit()

    def view_logs(self):
        """Displays all logged events."""
        self.db_manager.view_logs()

def login_user(db_manager):
    """Handles user login and returns the logged-in user."""
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")
    user = db_manager.get_user_by_username(username)

    if user and user.check_password(password):
        return user
    return None

def main():
    # Initialize database manager and logger
    db_manager = DatabaseManager()
    logger = Logger(db_manager)

    # Display a welcome message
    print("Welcome to the Secure Enclave Application!")
    print("Please log in to continue.")

    # Create admin and regular users
    admin_username = 'admin'
    admin_password = getpass.getpass("Enter admin password: ")
    regular_username = 'regular'
    regular_password = getpass.getpass("Enter regular user password: ")

    admin = Administrator(None, admin_username, '', UserRole.ADMIN)
    admin.set_password(admin_password)

    try:
        db_manager.store_user(admin)
    except sqlite3.IntegrityError:
        print("Admin username already exists. Choose a different username.")
        db_manager.close()
        return

    regular_user = RegularUser(None, regular_username, '', UserRole.REGULAR_USER)
    regular_user.set_password(regular_password)

    try:
        db_manager.store_user(regular_user)
    except sqlite3.IntegrityError:
        print("Regular user username already exists. Choose a different username.")
        db_manager.close()
        return

    while True:
        user = login_user(db_manager)
        if not user:
            print("Login failed.")
            continue

        while True:
            print(f"Logged in as {user.username} with role {user.get_role()}")
            if user.get_role() == UserRole.ADMIN:
                action = input("Choose action: [upload, delete, view_logs, logout, exit]: ")
                if action == 'upload':
                    file_path = input("Enter full path to file to upload: ")
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as file:
                                content = file.read()
                                artefact = Artefact(None, os.path.basename(file_path), content, user.get_user_id())
                                encryption_manager = EncryptionManager()
                                artefact.encrypt_content(encryption_manager)
                                db_manager.store_artefact(artefact)
                                logger.log_event("Uploaded artefact", user_id=user.get_user_id(), artefact_id=artefact.artefact_id)
                                print(f"Uploaded artefact {artefact.title}")

                                # Show artifact details
                                print("Artifact Details:")
                                print(f"ID: {artefact.artefact_id}")
                                print(f"Title: {artefact.title}")
                                print(f"Checksum: {artefact.checksum}")
                                print(f"Timestamp: {artefact.timestamps}")
                        except Exception as e:
                            print(f"Error reading file: {e}")
                    else:
                        print("File does not exist.")

                elif action == 'delete':
                    artefact_id = int(input("Enter artefact ID to delete: "))
                    db_manager.delete_artefact(artefact_id)
                    logger.log_event("Deleted artefact", user_id=user.get_user_id(), artefact_id=artefact_id)
                    print("Artefact deleted.")

                elif action == 'view_logs':
                    logger.view_logs()

                elif action == 'logout':
                    print("Logged out.")
                    break

                elif action == 'exit':
                    print("Exiting application.")
                    db_manager.close()
                    return  # Use return to exit from main loop without raising SystemExit

                else:
                    print("Invalid action.")

            elif user.get_role() == UserRole.REGULAR_USER:
                action = input("Choose action: [upload, modify, view, logout, exit]: ")
                if action == 'upload':
                    file_path = input("Enter full path to file to upload: ")
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as file:
                                content = file.read()
                                artefact = Artefact(None, os.path.basename(file_path), content, user.get_user_id())
                                artefact.encrypt_content(EncryptionManager())
                                db_manager.store_artefact(artefact)
                                logger.log_event("Uploaded artefact", user_id=user.get_user_id(), artefact_id=artefact.artefact_id)
                                print(f"Uploaded artefact {artefact.title}")

                                # Show artifact details
                                print("Artifact Details:")
                                print(f"ID: {artefact.artefact_id}")
                                print(f"Title: {artefact.title}")
                                print(f"Checksum: {artefact.checksum}")
                                print(f"Timestamp: {artefact.timestamps}")
                        except Exception as e:
                            print(f"Error reading file: {e}")
                    else:
                        print("File does not exist.")

                elif action == 'modify':
                    artefact_id = int(input("Enter artefact ID to modify: "))
                    existing_artefact = db_manager.retrieve_artefact(artefact_id)
                    if existing_artefact and existing_artefact.owner_id == user.get_user_id():
                        file_path = input("Enter full path to file to modify: ")
                        if os.path.exists(file_path):
                            try:
                                with open(file_path, 'rb') as file:
                                    content = file.read()
                                    artefact = Artefact(artefact_id, os.path.basename(file_path), content, user.get_user_id())
                                    artefact.encrypt_content(EncryptionManager())
                                    db_manager.update_artefact(artefact)
                                    logger.log_event("Modified artefact", user_id=user.get_user_id(), artefact_id=artefact_id)
                                    print(f"Modified artefact {artefact.title}")

                                    # Show updated artifact details
                                    print("Updated Artifact Details:")
                                    print(f"ID: {artefact.artefact_id}")
                                    print(f"Title: {artefact.title}")
                                    print(f"Checksum: {artefact.checksum}")
                                    print(f"Timestamp: {artefact.timestamps}")
                            except Exception as e:
                                print(f"Error reading file: {e}")
                        else:
                            print("File does not exist.")

                elif action == 'view':
                    artefact_id = int(input("Enter artefact ID to view: "))
                    artefact = db_manager.retrieve_artefact(artefact_id)
                    if artefact:
                        print(f"Artefact: {artefact.title}, Checksum: {artefact.checksum}, Timestamp: {artefact.timestamps}")

                elif action == 'logout':
                    print("Logged out.")
                    break

                elif action == 'exit':
                    print("Exiting application.")
                    db_manager.close()
                    return  # Use return to exit from main loop without raising SystemExit

                else:
                    print("Invalid action.")

if __name__ == "__main__":
    main()
